In [116]:
import numpy as np
import math

from sympy import Matrix
from sympy.physics.quantum import TensorProduct

In [117]:
# Finds the index where a linear system is violated, returns the row which is in violation

def violation(A,b,x):
    check = (A@x >= b)
    
    for box in range(len(check)):
        if check[box][0] == False:
            return box

In [181]:
# Gives a new center for one iteration of the ellipsoid method

def recenter(n, a, D, center):
    #print(a)
    a_horz = a
    a_vert = np.reshape(a, (int(n),-1))
    b = a_horz@D@a_vert
    
    center = center + (1/(n+1))*(1/math.sqrt(b[0]))*(D@a_vert)

    return center

In [182]:
# Gives a matrix defining the new shape of the ellipsoid after one iteration of ellipsoid method

def reshape(n, D, a):
    a_horz = a
    a_vert = np.reshape(a, (int(n),-1))
    b = a_horz@D@a_vert
    mid = TensorProduct(a_vert,a_horz)
    #print(mid)
    
    shape = (math.pow(n,2)/(math.pow(n,2)-1)) * (D - (2/(n+1))* ((D@mid@D)/b[0]) )
    #print(shape, '\n \n')
    
    return shape

In [183]:
# Goes through one iteration of the main loop of the ellipsoid method
# A and b define the polygon P we are testing for, P := {x: Ax >= b}
# center and D define the ellipsoid searching for P
# n is the dimension of the system

def Ellipsoid_Iter(A,b,center,D,n):
    # Checks to see if current center is feasible
    test = (A@center >= b)
    
    # If center is feasible
    if np.all(test):
        return print(f'The point {center} is feasible!')
    
    # If center is infeasible
    else:
        index = violation(A,b,center)
        a = np.array(A[index])

        center = recenter(n, a, D, center)
        D = reshape(n, D, a)
        
        new_ellipsoid = [center, D]
        
    return new_ellipsoid

In [196]:
# Goes through the ellipsoid Method

def Ellipsoid(A,b):
    ### INITIALIZE ###
     # Largest input of A or b
    U = max(A.max(),b.max())
    # Dimension of the system
    n = float(len(A[0])) 
    
    # minimum possible volume of P
    small_v = np.power(n,-n) * np.power((n*U), (-n**2 * (n+1))) 
    # Maximum possible volume of P
    big_V = np.power(2*n,n) * np.power(n*U, n**2) 
    
    # Time Limit
    time_limit = math.ceil(2*(n+1) * math.log(big_V/small_v, 2)) 
    # Radius squared of first unit ball
    radius = big_V/math.pi 
    # Defining matrix of first ellipsoid
    D = radius * np.identity(int(n)) 
    # First center set at 0
    center = np.zeros((int(n),1)) 
    
    time = 0
    data = [center,D]
    counter = 0
    
    ### MAIN LOOP ###
    while time != time_limit:
        # Complete One Iteration of Ellipsoid Check
        data = Ellipsoid_Iter(A,b,center,D,n)
        
        # If data type isn't a list, then we end the loop
        if type(data) != type([1,2,3]): 
            return(f'Process Ended in {counter} Iterations')
        
        # If data is a list, recenter and reshape
        center = data[0]
        D = data[1]
        time = time+1
        counter = counter+1
    
    return print(f'P is empty. Process ended in {counter} iterations')

In [198]:
A = np.array([[1,0],
              [-1,0],
              [0,1],
              [0,-1],
              [1,1]])

b = np.array([[2],
              [-3],
              [1],
              [-2],
              [-3]])

Ellipsoid(A,b)

The point [[2.22889712]
 [1.92743038]] is feasible!


'Process Ended in 12 Iterations'